# Visualizing and Understanding Neural Models in NLP  

## Abstract  

虽然神经网络已经成功应用于许多 NLP 任务，但由此产生的基于向量的模型很难解释。例如，他们如何实现组合性并不清楚，从单词和短语的含义来构建句子意义。在本文中，我们描述了用于 NLP 的神经模型中的组合性可视化的策略，其灵感来源于类似的计算机视觉工作。我们首先绘制 unit values ，用以表示negation,，强化和 concessive clauses 的组合性，从而使我们能够在 negation, 中看到众所周知的标记不对称。然后，我们介绍一些单元显著性的可视化方法，即从一阶导数对最终构成含义的贡献量。我们的通用方法在理解深度网络的组合性和其他语义属性方面可能有广泛的应用。  

## 1 Introduction  

神经模型在各种 NLP 任务中匹配或超越其他最先进系统的性能。然而，与传统的基于特征的分类器不同，这些分类器对各种可解释特征（词类，命名实体，单词形状，句法分析特征等）进行赋值和优化，深度学习模型的行为不太容易解释。深度学习模型主要通过多层神经结构对词嵌入（低维，连续，实值向量）进行操作，其每一层都被表征为一组隐藏的神经元单元。目前还不清楚深度学习模型是如何处理 composition,，实现 negation or intensification 等功能，或者结合句子不同部分的意义，建立句子意义。  

在本文中，我们探索了多种策略来解释神经模型中的意义构成。我们采用传统方法如表示性绘图，并引入简单的策略来测量神经单元对意义构成的贡献程度，它的“显著性”或使用一阶导数的重要性。  

本工作中呈现的可视化技术/模型揭示了神经模型的工作原理：例如，我们说明 LSTM 的成功是因为它能够比其他模型更加关注重要的关键词；多个composition 中的构成具有竞争性，并且这些模型能够捕捉负面的不对称，这是在自然语言理解中构成语义的重要属性；存在尖锐的空间局部性，某些维度以令人惊讶的定位方式标记否定和量化。尽管我们的尝试只涉及神经模型中的 superficial points 问题，并且每种方法都有其优缺点，但它们可以提供一些洞察神经模型在基于语言的任务中的行为，标志着在自然语言处理中理解它们如何实现有意义的 composition。  

下一节将介绍一些视觉和自然语言处理中的可视化模型，这些启发了这项工作。我们在第 3 节描述数据集和采用的神经模型。不同的可视化策略和相应的分析结果分别在第 4,5,6 节中给出，接下来是一个简短的结论。  

## 2 A Brief Review of Neural Visualization  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/2.jpg?raw=true)

通常通过将嵌入空间投影到两个维度并观察到类似的词倾向于聚集在一起，相似性通常通过图形可视化。（Karpathy et al。，2015）试图从统计观点来解释 recurrent 神经模型。其他相关尝试包括（Fyshe 等，2015；Faruqui 等，2015）。  

用于解释和可视化神经模型的方法在视觉中得到了更为重要的探索，尤其是对于卷积神经网络（CNN 或 ConvNets）（Krizhevsky等，2012），其中图像像素的原始矩阵被多层神经网络卷积并汇集到隐藏层。这些方法包括：  

（1）反演：通过训练一个额外的模型来反演表征，从不同的神经层次反馈到初始输入图像。重建背后的直觉是，可以从当前表示重构的像素内容。反演算法允许当前表示与原始图像的相应部分对齐。  

（2）反向传播和 Deconvolutional Networks：错误从输出层反向传播到每个中间层，最后传播到原始图像输入。反卷积网络以类似的方式工作，将输出逐层投影回初始输入，每一层与一个监督模型相关联，用于将较高的映射投影到较低的映射。这些策略使得有可能发现活跃区域或对最终贡献最大的区域。  

（3）生成：这组作品根据已经训练过的神经模型指导的草图生成特定类别的图像。模型以随机初始化的图像开始。在图像构建的不同阶段激活的特定图层可以帮助解释。  

尽管上述策略激发了我们在本文中提出的工作，但与 NLP 之间存在根本性差异。在 NLP 中词作为基本单位，因此（单词）向量而不是单个像素是基本单位。词的序列（例如，短语和句子）也以比像素的排列更结构的方式呈现。与我们的研究并行，Karpathy 等通过分析循环神经模型的预测和误差，从错误分析的角度探索了类似的方向。  

## 3 Datasets and Neural Models  

我们研究了两个神经模型训练的数据集，其中一个数据集规模相对较小，另一个规模较大。  

### 3.1 Stanford Sentiment Treebank  

斯坦福大学 Sentiment Treebank 是广泛用于神经模型评估的基准数据集。数据集包含每个分析树组件的 gold-standard  情绪标签，从句子到短语到单个词，用于 11855 个句子中的 215154 个短语。任务是在短语和句子两个层次上执行细粒度（非常积极，积极，中性，消极和非常消极）和粗粒度（积极与消极）分类。有关数据集的更多详细信息，请参阅 Socher et al。  

虽然这个数据集的许多研究都使用递归分析树模型，但在这项工作中，我们只使用标准序列模型（RNN 和 LSTM），因为这些模型是当前最广泛使用的神经模型，并且顺序可视化更直接。因此，我们首先将每个分析树节点转换为一个 tokens 序列。该序列首先映射到短语/句子表示并馈入 softmax 分类器。短语/句子表示使用以下三种模式构建：具有 TANH 激活函数的标准 Recurrent 序列，LSTM 和双向 LSTM。 有关这三种模型的详细信息，请参阅附录。  

### 3.2 Sequence-to-Sequence Models  

seq2seq 是旨在生成给定输入的输出文本序列的神经模型。从理论上讲，seq2seq 模型可以适应 NLP 任务，这些任务可以为给定输入的预测输出，并且由于不同的输入和输出而用于不同的目的，例如机器翻译，其输入对应于源语句和输出到目标语句；如果输入对应于消息并且输出对应于响应，则可用于对话。seq2seq 需要接受大量数据的训练，以获得学习对之间的隐式语义和句法关系。  

seq2seq 模型使用 LSTM 模型将输入序列映射到向量表示，然后基于预先获得的表示顺序预测 tokens。该模型定义了输出（Y）上的分布，并使用 softmax 函数依次预测给定输入（X）的 tokens。  

## 4 Representation Plotting  

我们从简单的 plots 开始，使用斯坦福情绪树库（Stanford Sentiment Treebank）揭示 local compositions。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/1.jpg?raw=true)

**Local Composition** 图1 显示了一个 60d 热图矢量，用于表示选定的单词/短语/句子，重点在于 extent 修改（状语和 adjectival）和 negation。短语或句子的嵌入是通过组合来自预训练模型的词表达来实现的。  

我们在图2 中使用 tsne（Van der Maaten和Hinton，2008）对单词和短语进行可视化，故意添加一些随机单词以供比较。可以看出，神经模型很好地学习了 local compositionally 性质，将否定词+积极词（“不好”，“不好”）与负面词组合在一起。还要注意否定的不对称性：“not bad”更多的和负性词在聚在一起（如图1和图2所示）。这种不对称性在语言学中得到了广泛的讨论，例如由于显著性而引起的，因为'好'是该尺度的未标记方向。这表明，虽然模型在图1 中似乎集中于某些 negation，但神经模型不仅仅是学习将固定变换应用于“不”，而是能够捕捉不同单词构成中的细微差异。  

**Concessive Sentences ** 在让步性句子中，两个从句具有相反的词性，通常与期望相反的含义相关。图3 绘制出两个让步从句随时间演变的表示图。这些图表明：  

1.对于目标是预测特定语义维度（与语言模型词预测等一般任务相反）的情感分析等任务而言，过大的维度会导致许多维度无法运行（值接近0），从而导致两个维度 相反情绪的句子仅在几个维度上有所不同。 这可以解释为什么更多的维度不一定会导致这些任务更好的表现（例如，如（Socher等人，2013）中所报道的，当词维度设置在25和35之间时达到最佳性能）。  

。。。  

**Clause Composition** 在图4 中，我们更详细地探讨了此 clause composition。通过添加负面 clauses，比如“although it had bad acting”，或者““but it is too long”到简单的积极的“I like the movie”的结尾，这些表现力更接近负面情绪区域。相比之下，为负面 clause 增加一个concessive clause 并不会朝着积极的方向发展；“I hate X but ...”仍然是非常消极的，与“I hate X”没有什么不同。这种差异再次表明该模型能够捕捉负面不对称。  

## 5 First-Derivative Saliency  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/3.jpg?raw=true)

在本节中，我们描述了另一种策略，它受到视觉中的后向传播策略的启发。它衡量每个输入单位对最终决策的贡献量，可以用一阶导数来近似。  

更正式地说，对于分类模型，输入 E 与标准类别标签 c 相关联。（根据NLP任务，输入可以是单词或单词序列的嵌入，而标签可以是 POS 标签，情感标签，预测的下一个单词索引等）给定输入词的嵌入 E 以及关联标签 c，训练后的模型将该对（E，c）与 $S-c(E)$ 分数联系起来。目标是决定 E 的哪些units 对 $S-c(E)$ 做出最重要的贡献，并因此决定是否选择类别标签 c。   

### 5.1 Results on Stanford Sentiment Treebank  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/4.jpg?raw=true)

我们首先在斯坦福树库上说明结果。我们在图5，图6 和图7 中绘制了三个句子的显著性分数，将训练的模型应用于每个句子。每行对应于表示每个维度的每个网格的对应词表示的显著性分数。这些例子是基于明确的情绪指标“hate”，这些指标给了他们所有的负面情绪。  

**“I hate the movie”** 所有这三种模式都高度赞同“hate”，并削弱了其他 tokens 的影响力。LSTM 比标准的循环模型更加关注“hate”，但双向LSTM 显示最清晰的焦点，几乎不重视“hate”以外的单词。这大概是由于 LSTM 和 Bi-LSTM 中的门结构控制信息流，使得这些体系结构更好地滤除不太相关的信息。  

**“I hate the movie that I saw last night”** 所有三种模式都指定了正确的观点。简单的循环模型在筛选出不相关的信息方面再次做得很差，对与情绪无关的单词分配过多的显著性。然而，尽管这个句子更长，但没有一个模型受到梯度消失问题的影响；“hate”的显著性在卷积操作之后仍然突出。  

**““I hate the movie though the plot is interesting”** 简单的循环模型只强调第二个 clause “the plot is interesting”，并没有对第一个 clause “I hate the movie”给予评价。这看起来可能是由渐变消失引起的，但该模型正确地将该句子分类为非常消极，表明它成功合并来自第一个否定句子的信息。我们单独测试了单个 clause “though the plot is interesting”。标准循环模型自信地将其标记为正面。因此，尽管第一个句子中的单词显著性得分较低，但简单的循环系统设法依赖该句子，并且从后面的肯定句子中淡化信息 - 尽管后面的单词具有较高的显著性分数。这说明了显著性可视化的局限性。一阶导数不能捕获所有我们想要显示的信息，也许是因为它们只是粗略地接近 individual 贡献，并且可能不足以处理高度非线性的情况。相比之下，LSTM 强调第一个 clause，大大减弱了第二个 clause 的影响，而 Bi-LSTM 侧重于“hate the movie”和“plot is interesting”。  

### 5.2 Results on Sequence-to-Sequence Autoencoder  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/6.jpg?raw=true)

图9 表示在每个时间步骤预测对应 token 方面自动编码器的显著热图。随着解码的进行，我们通过反向传播计算每个前一个词的一阶导数。每个网格对应于每个 1000 维单词向量的平均显著性值的大小。热图给出了解码过程中神经模型行为的清晰概览。观察总结如下：  

1.对于每个预测单词的时间步长，seq2seq 模型设法将单词预测返回到输入端的对应区域（自动学习对齐），例如，当需要预测的 token 为“hate”时，以 token “hate”为中心的输入区域会产生更大的影响。  

2.神经解码将先前构建的表示与当前步骤中预测的单词相结合。随着解码的进行，初始输入对解码（即源句子中的tokens）的影响逐渐减小，因为更多的预先预测的单词被编码在矢量表示中。与此同时，语言模型的影响逐渐占据主导地位：当预测单词“boring”时，模型更重视早期预测 tokens "plot" 和 "is"，但对输入中对应的区域较少关注，eg. 输入中的单词“boring “。  

## 6 Average and Variance  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/1/5.jpg?raw=true)

对于将单词嵌入视为参数以从头开始优化的设置（与使用预先训练的嵌入相反），我们提出了第二个令人惊讶的简单而直接的方法来可视化重要指标。我们首先计算句子中所有单词的单词嵌入的平均值。一个词的显著性或影响力的衡量标准是它偏离这个平均值。这个想法是，在训练过程中，模型会学习使 indicators 与 non-indicator 词不同，即使在经过多层计算之后，它们也能够脱颖而出。  

图8 显示了方差图；每个网格对应于 $\parallel e_{ij} - \frac {1}{N_s} \sum_{i' \in N_S} e_{i'j} \parallel^2$ 的值，其中 $e_{ij}$ 表示单词 i 的第 j 维的值，N 表示句子内 token 的数量。  

如图所示，基于方差的显著性度量也很好地强调了相关的情感词。该模型确实有缺点：（1）它只能用于字嵌入是学习参数的场景（2）很明显，模型能够很好地显示局部组合性。  

## 7 Conclusion  

在本文中，我们提供了几种方法来帮助可视化和解释神经模型，了解神经模型如何构成 meanings，展示 negation 的不对称性并在某些方面解释 LSTM 在这些任务中的强大性能。  

尽管我们的尝试只是触及神经模型中的表面观点，并且每种方法都有其优点和缺点，但它们可以一起提供有关神经模型在基于语言的任务中的行为的一些见解，标志着理解它们如何在自然语言处理获得意义组成的初始步骤。我们未来的工作包括使用可视化结果来执行错误分析，以及了解不同神经模型的强度限制。  

# Distributed Representations of Words and Phrases and their Compositionality  

## Abstract  

最近引入的连续 Skip-gram 模型是学习高质量 distributed vector representations(分布向量表示)的有效方法，distributed vector representations可以捕获大量精确的句法和语义关系。在本文中，我们提出了几个扩展，提高了向量的质量和训练速度。通过对 frequent words 进行二次抽样，我们获得了显著的加速，同时还学习了更多的 regular word representations(常规单词表示)。我们还提出了一个分层 softmax 的简单替代方案，称为 negative sampling (负采样)。word representations 的一个固有限制是：它们不关心词序，而且无法表示 idiomatic phrases(习惯用语)。例如，不能简单地将“Canada/加拿大”和“Air/空中”的含义组合起来得到“Canada Air/加拿大航空公司”的含义。在这个例子的启发下，我们提出了一种在文本中查找短语的简单方法，并表明学习数百万个 phrases 的 good vector representations 是可能的。  

## 1 Introduction  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/1.jpg?raw=true)

通过分组相似的单词，在向量空间中的 distributed representations 可以帮助学习算法在 NLP 任务中获得更好的表现。最早使用 word representations 可以追溯到 1986 年(Rumelhart，Hinton 和 Williams)。这个想法已经被应用于统计语言建模且取得了相当大的成功。后续工作包括应用于自动语音识别和机器翻译，以及大范围的 NLP 任务。  

最近，Mikolov 等人引入了 Skip-gram 模型，这是一种从大量非结构化文本数据中学习高质量向量表示的有效方法。与过去大部分用于学习 word vectors 的神经网络架构不同，Skip-gram 模型的训练(参见图1)不涉及密集矩阵的乘法。这使得训练非常高效：一个优化过的单机实现可以在一天内训练超过 1000 亿字。  

使用神经网络计算的 word representation 非常有趣，因为已训练的向量明确地编码了许多语言规律和模式。有点令人惊讶的是，许多这些模式可以表示为线性变换。例如，向量计算 vec("Madrid") - vec("Spain") + vec("France") 的结果比任何其他 word vector 更接近于 vec("Paris")。   

在本文中，我们提出了原始 Skip-gram 模型的几个扩展。在训练过程中，对 frequent words 进行二次采样会导致显著的加速（大约2-10倍），并提高频率较低的 word representation 的准确性。此外，我们提出了一种用于训练 Skip-gram 模型的简化 NCE(Noise Contrastive Estimation/噪声对比估计)。结果表明，与更复杂的分层 softmax 相比，它有更快的训练速度，而且 frequent words 的 vector representation 也更好。   

words representation 天生受限于 idiomatic phrases 的表示。例如，“Boston Globe/波士顿环球报”是报纸，它不是“Boston/波士顿”和“Globe/地球”的含义的自然组合。因此，用向量来表示整个短语会使 Skip-gram 模型更具表现力。其他旨在通过组合单词向量（例如递归自动编码器/recursive autoencoders）来表示句子意义的技术也将受益于使用 phrase vectors 而不是 word vectors。   

模型从基于单词扩展到基于短语模型相对简单。首先，我们使用 data-driven 的方法识别大量的短语，然后在训练过程中将短语视为单独的 tokens (标记)。为了评估短语向量的质量，我们开发了一个包含单词和短语的类比推理任务测试集。测试集中一个典型类比对是 "Montreal":"Montreal Canadiens" :: "Toronto":"TorontoMaple Leafs" 如果最靠近 vec("Montreal Canadiens") - vec("Montreal") + vec("Toronto") 的表达是 vec("Toronto Maple  Leafs")，则被认为回答正确。   

最后，我们描述了 Skip-gram 模型的另一个有趣属性。我们发现简单的向量加法通常可以产生有意义的结果。例如，vec("Russia") + vec("river") 接近  vec("Volga River")，而 vec("Germany") + vec("capital") 接近 vec("Berlin")。这种组合性表明，通过对 word vector representation 使用基本的数学运算，可以获得不明显(non-obvious)程度的语言理解。  

## 2 The Skip-gram Model  

Skip-gram 模型的训练目标是找到可用于预测句子或文档中 surrounding words 的 word representation。更正式地，给出训练词 $w_1,w_2,,...,w_T$，Skip-gram 模型的目标是最大化对数概率：   

$\frac {1}{T} \sum_{t=1}^{T} \sum_{-c \leq j \leq c, j \neq 0} \log p(w_{t+j}|w_t) \tag{1}$

其中 c 是训练上下文（可以是中心单词 $w_t$ 的一个函数）的大小。较大的 c 意味着更多的训练样本，因此可以导致更高的准确性，同时也意味着更多的训练时间。基本 Skip-gram 公式使用 softmax 函数定义 $p(w_{t+j}|w_t)$:   

$p(w_O|w_I) = \frac{\exp (v'^T_{w_O} v_{w_I})}{\sum_{w=1}^W \exp (v'^T_{w_O} v_{w_I})} \tag{2}$

其中 $v_w$ 和 $v'_w$ 分别为 w 的输入和输出向量表示，W 为词汇表中的单词数。这个公式是不切实际的，因为计算 $\Delta \log p(w_O|w_I)$ 的花费与 W 成正比，通常会达到 $10^5 - 10^7$ 的数量级。  

### 2.1 Hierarchical Softmax  

Hierarchical(分层) softmax 是完全 softmax 的近似有效计算。它首先由 Morin 和 Bengio 在神经网络语言模型的上下文中引入。它的主要优点是，不需要评估神经网络中的 W 个输出节点以获得概率分布，仅需要评估约 log2(W) 个节点。  

分层 softmax 使用二叉树表示输出层，其中 W 个字作为其叶子节点，并且对于每个节点，显式地表示其子节点的相对概率。这些定义了一个可将概率分配给单词的 random walk(随机游走)。  

更准确地说，从一条合适的路径，可以从 root 到达每个单词 w。设 $n(w,j)$ 为从 root 到单词 w 的路径上的第 j 个节点，L(w) 为该路径的长度，则 $n(w,1) = root, n(w,L(w)) = w$。另外，对每个内节点，设 ch(n) 为 n 的 arbitrary fixed child，如果 x 是 true 则 [x]=1，否则 [x]=-1。则分层softmax 将按照如下公式定义 $p(w_O|w_I)$:   

$p(w|w_I) = \prod_{j=1}^{L(w)-1} \sigma ([n(w, j + 1) = ch(n(w, j))] \cdot v'^T_{n(w,j) v_{wI}}) \tag{3}$

其中 $\sigma(x) = 1/(1+\exp(-x))$。可以证明的是 $\sum_{w=1}^W p(w|w_I) = 1$。这意味着计算 $\log p(w_O|w_I)$ 和 $\Delta \log p(w_O|w_I)$ 的消耗与 $L(w_O)$ 成正比，通常来说不超过 $\log W$。此外，不像 Skip-gram 的标准 softmax 公式会把两个表示 $v'_w$ 和 $v_w$ 分配给每个单词 w，在分层 softmax 公式中每个单词 w 有一个 $v_w$ 且二叉树的每个内部节点 n 有一个 $v'_n$。   

分层 softmax 使用的树结构对性能有相当大的影响。Mnih 和 Hinton 探索了构建树结构的一些方法以及训练时间和结果模型精度的影响。在我们的工作中，我们使用一个霍夫曼树（binary Huffman tree），因为它将短 codes 分配给高频词，从而加快了训练速度。之前已经观察到，根据出现频率组合单词可以很好的作为基于神经网络的语言模型的一种简单加速技术。  

### 2.2 Negative Sampling  

分层 softmax 的替代方案是噪声对比估计（NCE），由 Gutmann 和 Hyvarinen 引入，并由 Mnih 和 Teh 用于语言建模。NCE 认为一个好的模型应该能够通过 logistic regression 来区分数据和噪声。这类似于 Collobert 和 Weston 使用的折页损失/hinge loss，他们通过对噪声上的数据进行排名来训练模型。  

虽然 NCE 可以最大化 softmax 的对数概率，但是 Skip-gram 模型只关注学习高质量的向量表示，因此只要向量表示保持其质量，我们可以随意简化 NCE。我们通过以下公式定义 Negative Sampling（NEG）:   

$\log \sigma(v'^T_{w_O} v_{w_I}) + \sum_{i=1}^k E_{w_i - P_n(w)} [\log \sigma(-v'^T_{w_i} v_{w_I})] \tag{4}$

用于替换 Skip-gram 中的每个 $\log p(w_O|w_I)$ 项。因此，任务是使用逻辑回归来从噪声分布 $P_n(w)$ 中区分目标单词 WO，其中每个数据样本有 k 个负样本。我们的实验表明，k 值在 5-20 范围内对于小型训练数据集是有用的，而对于大型数据集，k 可以小至 2-5。负采样和 NCE 的主要区别在于 NCE 需要样本和噪声分布的数字概率，而负采样只使用样本。尽管 NCE 使 softmax 的对数概率近似最大化，但这个属性对于我们的应用并不重要。  

NCE 和 NEG 均具有作为自由参数的噪声分布 $P_n(w)$。我们发现 unigram distribution U(w) 可以提高 NCE 和 NEG 的性能。   

### 2.3 Subsampling of Frequent Words  

在非常大的语料库中，最常见的单词可能容易出现数亿次（例如“in”，“the”和“a”）。这些单词通常比罕见单词提供更少的信息价值。例如，虽然 Skip-gram 模型受益于观察“France”和“Paris”的共现，但由于观察到“France”和“the”的频繁共现，因此几乎每一个字 - 在“the”的句子中频繁发生。这个想法也可以用于相反的方向；训练数百万个例子后，频繁词的向量表示没有显著变化。  

为了解决罕见词和常见词之间的不平衡问题，我们使用了一种简单的子采样方法：训练集中的每个词 wi 按以下公式计算的概率被丢弃  

$P(w_i) = 1 - \sqrt {\frac{t}{f(w_i)}} \tag{5}$

其中 $f(w_i)$ 是单词 wi 的频率，t 是一个选定的阈值，一般在 $10^5$ 左右。我们选择这个子采样公式是因为它在保留频率排序的同时积极地抽样频率大于 t 的词。虽然这个子采样公式是启发式选择的，但我们发现它在实践中运作良好。它会加速学习，甚至可以显著提高罕见单词的学习向量的准确性，这将在以下各节中介绍。  

## 3 Empirical Results  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/2.jpg?raw=true)

在本节中，我们评估的 Hierarchical Softmax（HS），噪声对比估计，Negative Sampling 和训练单词子采样。我们使用 Mikolov 等人介绍的类比推理任务。该任务由类似于“Germany” : “Berlin” :: “France” : ? 的类比组成，通过找到一个向量 x 来解决，vec（x）的余弦距离最接近 vec(“Berlin”) - vec(“Germany”) + vec(“France”)。如果 x 是“Paris”，这个特定的例子被认为已经被正确回答。该任务有两大类：句法类比（如“quick” : “quickly” :: “slow” : “slowly”）和语义类比，如从国家到首都的关系。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/3.jpg?raw=true)

为了训练 Skip-gram 模型，我们使用了一个由各种新闻文章组成的大型数据集（一个 Google 内部数据集，有十亿字）。我们在训练数据中丢掉了所有出现少于 5 次的单词，结果产生了一个大小为 692K 的词汇。表1 中报告了各种 Skip-gram 模型对词类比测试集的性能。该表显示 Negative Sampling 在类比推理任务上优于 Hierarchical Softmax，并且性能甚至比噪声对比估计略好。频繁词的二次采样提高了几倍的训练速度，并使词表达更加准确。  

可以说，skip-gram 模型的线性使得它的向量更适合于这种线性类比推理，但 Mikolov 等人[8]也表明，随着训练数据量的增加，由标准 S 形回归神经网络（高度非线性）学习的向量显著改善了这一任务，这表明非线性模型对线性结构的 word representations 也有帮助。  

## 4 Learning Phrases  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/4.jpg?raw=true)

正如前面所讨论的，许多短语的含义并不是单个单词含义的简单组合。为了学习短语的矢量表示，我们首先找到经常出现在一起的单词，并且很少出现在其他上下文中。 例如，“纽约时报”和“多伦多枫叶队”在训练数据中被独特的 tokens 所取代，而“this is”这个 bigram 将保持不变。  

这样，我们可以形成很多合理的短语，而不会大大增加词汇的大小；从理论上讲，我们可以使用所有的 n 元语言来训练 Skip-gram 模型，但是这会占用过多的内存。之前已经开发了许多技术来识别文本中的短语；然而，我们的工作范围不在于比较它们。我们决定使用一种简单的数据驱动方法，在这种方法中，短语是基于 unigram 和 bigram 计数形成的  

$score(w_i,w_j) = \frac {count(w_i,w_j) - \delta}{count(w_i) \times count(w_j)} \tag{6}$

δ被用作 discounting 系数，并且防止形成包含非常罕见单词的太多短语。然后将具有高于所选阈值的分数用作短语。通常，我们在阈值下降的情况下对训练数据运行 2-4 次，允许形成由多个单词组成的更长的短语。我们使用涉及短语的新类比推理任务来评估短语表示的质量。表 2 显示了这个任务中使用的五类比喻的例子。该数据集可在 Web2 上公开获得。  

### 4.1 Phrase Skip-Gram Results  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/5.jpg?raw=true)

从与之前实验相同的新闻数据开始，我们首先构建了基于短语的训练语料库，然后我们使用不同的超参数训练了多个 Skip-gram 模型。与之前一样，我们使用矢量维度 300 和上下文大小 5。此设置已经在短语数据集上实现了良好的性能，并且允许我们快速比较 Negative Sampling 和 Hierarchical Softmax，无论是否使用 frequent tokens 的 subsampling。结果总结在表3 中。  

结果表明，虽然负采样在 k = 5 时仍能达到可观的精度，但使用 k = 15 可以获得相当好的性能。令人惊讶的是，虽然我们发现 Hierarchical Softmax 在没有二次采样的情况下训练时性能较低，但是当我们对频繁字进行下采样时，它成为了表现最佳的方法。这表明子采样可以导致更快的训练，并且还可以提高准确性，至少在某些情况下。  

为了最大化词组类比任务的准确性，我们通过使用大约 330 亿字的数据集来增加训练数据量。我们使用 hierarchical softmax，维度为 1000，整个句子为上下文。这导致了一个达到 72% 准确度的模型。当我们将训练数据集的大小减少到 6B 个单词时，我们的准确率降低了 66%，这表明大量的训练数据是至关重要的。  

为了进一步深入了解不同模型所表现出来的表征之间的差异，我们使用各种模型手动检查了不常见短语的最近邻居。在表4 ，我们展示了这种比较的一个样本。 与以前的结果一致，似乎短语的最佳表示是通过具有 hierarchical softmax 和二次采样的模型学习的。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/6.jpg?raw=true)

## 5 Additive Compositionality  

我们证明了由 Skip-gram 模型学习的单词和短语表示呈现出一种线性结构，可以使用简单的矢量算术执行精确的类比推理。有趣的是，我们发现 Skip-gram 表示展示了另一种线性结构，它可以通过 element-wise 方式添加它们的向量来表示有意义地组合词语。表5 说明了这种现象。  

通过检查训练目标可以解释向量的加性特性。单词矢量与 softmax 非线性的输入成线性关系。当词向量被训练来预测句子中的周围词时，向量可以被看作表示词出现的上下文的分布。这些值与由输出层计算的概率以对数形式相关，所以两个单词向量的总和与两个上下文分布的乘积有关。该乘积在这里用作 AND 函数：由两个单词向量分配高概率的单词将具有高概率，而其他单词将具有低概率。因此，如果“伏尔加河”与“俄罗斯”和“河流”一起经常出现在同一个句子中，这两个单词向量的总和将导致这样一个特征向量接近“伏尔加河”向量，。  

## 6 Comparison to Published Word Representations  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/2/7.jpg?raw=true)

许多以前从事基于神经网络词汇表征的作者已经发表了他们的结果模型供进一步的使用和比较：其中最着名的作者是 Collobert 和 Weston [2]，Turian ，Mnih 和 Hinton [10]。我们从 web3 下载了他们的单词向量。Mikolov 等人[8]已经评估了单词类比任务中的这些单词表示，其中 Skip-gram 模型以巨大的优势实现了最佳性能。  

为了更深入地了解学习向量质量的差异，我们通过显示表6 中不常用词的最近邻居来提供经验性比较。这些例子表明，在大型语料库上训练的 Skip-gram 模型明显优于所有其他模型。这部分可以归因于这个模型已经被训练了大约 300 亿个单词，这比以前的工作中使用的典型大小多出大约两到三个数量级的数据。有趣的是，尽管训练集更大，但 Skip-gram 模型的训练时间仅仅是以前模型架构所需的时间复杂度的一小部分。  

## 7 Conclusion  

这项工作有几个关键贡献。我们展示了如何使用 Skip-gram 模型来训练单词和短语的分布式表示，并且证明这些表示呈现出可以进行精确的类比推理的线性结构。本文介绍的技术也可用于训练[8]中介绍的 bag-of-words 模型。  

由于采用了计算效率高的模型架构，我们成功地对比以前发布的模型多数量级的数据进行了模型训练。这导致学习的单词和短语表示的质量大大提高，特别是对于罕见的实体。我们还发现，频繁词汇的二次抽样导致更快的训练和更好地表达不常见的单词。我们的论文的另一个贡献是负采样算法，它是一种非常简单的训练方法，可以学习精确的表示，特别是对于频繁的词。  

训练算法和超参数选择的选择是一个任务特定的决定，因为我们发现不同的问题有不同的最优超参数配置。在我们的实验中，影响性能的最关键的决定是模型架构的选择，矢量的大小，子采样率以及训练窗口的大小。  

这项工作的一个非常有趣的结果是单词向量可以通过简单的向量添加而有意义地组合。学习本文中表达的短语的另一种方法是简单地用单个标记表示短语。 这两种方法的组合提供了一种强大而简单的方法，即如何表示更长的文本片段，同时具有最小的计算复杂度。因此，我们的工作可以被看作是试图用递归矩阵向量操作来表示短语的现有方法的补充[16]。  

我们根据本文描述的技术制定了用于训练词和短语向量的代码，可作为开源项目使用。

# Word2Vec Tutorial  

本教程介绍 Word2Vec 的 skip-gram 神经网络架构。本教程的目的是跳过关于 Word2Vec 的常见介绍性和抽象的见解，并深入了解更多细节。具体来说，我正在 diving into 神经网络模型。  

## The Model  

skip-gram 神经网络模型其最基本的形式实际上是非常简单的。

让我们从关于我们要做什么的高层次洞察开始。Word2Vec 使用了一个你可能在机器学习中看到过的技巧。我们将训练一个带有单个隐藏层的简单神经网络来完成特定的任务，但是我们实际上并没有将这个神经网络用于我们训练的任务！相反，目标实际上只是学习隐藏层的权重 - 我们会看到这些权重实际上是我们试图学习的“单词向量”。  

另一个你可能见过这个技巧的地方是无监督的特征学习，在这里你训练一个自动编码器来压缩隐藏层中的输入向量，并将其解压缩回输出层的原始数据。 训练完成后，您会剥离输出层（解压缩步骤），然后使用隐藏层 - 这是学习良好图像特征而不标记训练数据的诀窍。

## The Fake Task  

所以现在我们需要谈论这个我们将要建立神经网络来执行的“假”任务，然后我们会稍后再回到这个间接给我们提供那些我们真正需要单词向量的方法。

我们将训练神经网络来完成以下工作。给定一个句子中的特定单词（输入单词），查看附近的单词并随机选择一个单词。网络将告诉我们在我们的词汇中每个词是我们选择的“附近词”的可能性。

当我说“附近”时，算法实际上有一个“窗口大小”参数。一个典型的窗口大小可能是 5，意味着前后各有 5 个字（总共10个字）。
输出概率将与在我们的输入词附近找到每个词汇单词的可能性有关。例如，如果你给训练好的网络输入单词“苏联”，输出概率对于“联盟”和“俄罗斯”这样的单词来说要比“西瓜”和“袋鼠”这样无关的单词要高得多。

我们将通过输入我们的训练文档中找到的单词对来训练神经网络。下面的例子显示了一些训练样本（单词对），我们将从“The quick brown fox jumps over the lazy dog”一句中取出。我已经使用了一个 2 的小窗口大小作为例子。以蓝色突出显示的单词是输入单词。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/1.jpg?raw=true)

网络将从每次配对显示的次数中学习统计数据。例如，网络可能会获得更多（“苏联”，“联盟”）的训练样本，而不是（“苏联”，“萨斯奎奇”）的训练样本。训练结束后，如果您输入“苏联”一词作为输入，那么它将输出“联盟”或“俄罗斯”的概率高于“Sasquatch”的概率。  

## Model Details  

那么这是如何表现的？

首先，你知道你不能将一个单词作为一个文本字符串输入到神经网络，所以我们需要一种方法来将这些单词表达给网络。为此，我们首先根据训练文档建立一个单词词汇表 - 假设我们有一个包含 10,000 个独特词汇的词汇表。

我们将代表像“蚂蚁”这样的输入词作为一个 one-hot vector。这个向量将有 10,000 个 components，我们将在对应于单词“蚂蚁”的位置放置一个“1”，并在所有其他位置放置一个“0”。

网络的输出是一个单一的向量（也有 10,000 个 components），对于我们的词汇表中的每个单词，它表示该单词是输入单词附近词的概率。

这是我们神经网络的架构。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/2.jpg?raw=true)

隐层神经元没有激活函数，但输出神经元使用 softmax。我们稍后再回来。

当在单词对上训练这个网络时，输入是代表输入词的 one-hot 向量，并且训练输出也是代表输出词的 one-hot 向量。但是，当你在输入字上评估训练好的网络时，输出向量实际上是一个概率分布（即一组浮点值，而不是一个 one-hot 向量）。  

## The Hidden Layer  

对于我们的例子，我们要说我们正在学习 300 个特征的单词向量。因此，隐藏层将由一个 10,000 行（我们词汇表中的每个词）和 300 列（每个隐藏的神经元）的权重矩阵表示。

300 特征是 Google 在他们发布的 Google 新闻数据集训练模型中使用的特征数量。特征的数量是一个“超参数”，你只需要调整你的应用程序（即尝试不同的值，看看什么会产生最好的结果）。

如果你看这个权重矩阵的行，这些实际上是我们的词向量！  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/3.jpg?raw=true)

因此，所有这些的最终目标只是为了学习这个隐藏层的权重矩阵 - 当我们完成时我们会抛弃输出层！

但是，让我们回过头来看看我们要训练的这个模型的定义。

现在，你可能会问自己 - “这个 one-hot 向量几乎全是零...那有什么效果呢？”如果你将一个 1×10,000 one-hot 向量乘以一个 10,000×300 的矩阵，它将有效地选择矩阵行对应于“1”。这里有一个小例子给你一个视觉效果。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/4.jpg?raw=true)

这意味着此模型的隐藏层实际上只是作为查找表来操作。隐藏层的输出只是输入单词的“单词向量”。  

## The Output Layer  

“蚂蚁”的 1×300 word 向量然后被送到输出层。输出层是一个 softmax 回归分类器。这里有一个关于 Softmax 回归的深入教程，但其要点是每个输出神经元将产生一个介于 0 和 1 之间的输出，并且所有这些输出值的总和将加起来是 1。

具体来说，每个输出神经元都有一个权重向量，它与来自隐藏层的单词向量相乘，然后将函数 exp(x) 应用于结果。最后，为了使输出总和为 1，我们将这个结果除以所有 10,000 个输出节点的结果之和。

下面是计算单词“car”的输出神经元输出的图示。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/5.jpg?raw=true)

请注意，神经网络不知道输出词相对于输入词的偏移量。它并没有为输入之前的单词与之后的单词学习不同的一组概率。为了理解这个含义，假设在我们的训练语料库中，单词 'York' 的每一个单词前面都有 'New' 单词。也就是说，至少根据训练数据，'New' 将在 'York' 附近有 100% 的概率。然而，如果我们把 'York' 附近的 10 个单词随机选取其中的一个，那么 'New' 的概率不是100%。你可能选择了附近的其他单词之一。  

## Intuition  

好的，你准备好了解这个网络的一些情况吗？

如果两个不同的单词具有非常相似的“contexts”（也就是说，他们周围可能出现哪些单词），那么我们的模型需要为这两个单词输出非常相似的结果。 对于这两个词，网络输出类似上下文预测的一种方式是如果词向量相似。所以，如果两个词有相似的上下文，那么我们的网络就有动机学习这两个词的相似词向量！

两个词有相似的 contexts 是什么意思？ 我认为你可以期望像“智能”和“聪明”这样的同义词具有非常相似的 contexts。或者，与“引擎”和“传播”相关的词语可能也有类似的语境。

这也可以处理词干 - 网络可能会学习单词“ant”和“ants”的相似单词向量，因为它们应该有类似的上下文。  

## part 2

在 word2vec 教程的第 2 部分中，我将介绍对基本 skip-gram 模型的一些额外修改，实际上这些修改对于使其可以有效训练非常重要。

当您阅读 Word2Vec 的 skip-gram 模型教程时，您可能已经注意到了一些东西 - 这是一个巨大的神经网络！

在我给出的例子中，我们有 300 个 components 的单词向量和 10,000 个单词的词汇表。回想一下，神经网络有两个权重矩阵 - 隐藏层和输出层。这两层都会有一个权重矩阵，每个权重矩阵为 300 x 10,000 = 300 万！

在一个大的神经网络上运行梯度下降是很缓慢的。更糟糕的是，你需要大量的训练数据才能调整许多权重并避免过拟合。数百万的权重乘以数十亿训练样本意味着训练这个模型将成为一个野兽。

Word2Vec 的作者在他们的第二篇论文中讨论了这些问题。

第二篇论文有三个创新点：

- 1 在他们的模型中将常用单词对或短语视为单个“单词”。  

- 2 对频繁的词进行子采样以减少训练实例的数量。  

- 3 用他们称之为“负采样”的技术修改优化目标，这会使每个训练样本只更新一小部分模型的权重。

值得注意的是，对频繁词进行二次抽样和应用负抽样不仅减少了训练过程的计算负担，而且还提高了其结果词向量的质量。

## Word Pairs and “Phrases”  

作者指出，像“Boston Globe”（一家报纸）这样的词组与“Boston”和“Globe”这两个单词有着非常不同的含义。因此，将“Boston Globe”无论出现在文本中的任何地方都视为具有自己词汇向量表示的单个词汇，这是有道理的。

您可以在他们发布的模型中查看结果，该结果从 Google 新闻数据集中对 1000 亿字进行了训练。在模型中加入短语使词汇量增加到 300 万字！

如果你对他们所产生的词汇感兴趣，我可以稍微探讨一下。你也可以在这里浏览他们的词汇。

短语检测包含在他们的论文的“Learning Phrases”部分。他们在 word2phrase.c 中分享了他们的实现。

我不认为他们的短语检测方法是他们论文的一个关键贡献，但是我会分享一些，因为它非常简单。

他们的工具每次只查看 2 个单词的组合，但可以多次运行以获得更长的短语。因此，第一遍会选择“New_York”，然后再次运行它 将 “New_York_City” 选为“New_York”和“City”的组合。

该工具计算训练文本中两个单词的每个组合出现的次数，然后将这些计数用于等式中以确定将哪些单词组合成短语。它还喜欢使用不常见字组成的短语词组，以避免 “and the” 或“this is”等常见词组成词组。

我认为有一种替代词组识别策略，就是将所有维基百科文章的标题用作词汇。

## Subsampling Frequent Words  

在本教程的第1 部分中，我展示了如何从源文本创建训练样本，但我会在此重复。下面的例子显示了一些训练样本（单词对），我们将从“The quick brown fox jumps over the lazy dog”一句中取出。我已经使用了一个 2 的小窗口作为例子。以蓝色突出显示的单词是输入单词。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/3/1.jpg?raw=true)

用“the”这个常见词有两个“问题”：

- 1 当看单词对时，(“fox”，“the”)并没有告诉我们很多关于“fox”的含义。“the”几乎出现在每个单词的上下文中。  

- 2 我们将有比我们需要为“the”学习一个好的向量的更多的样本(“the”，...)。  

Word2Vec 实现了一个“subsampling”方案来解决这个问题。对于我们在训练文本中遇到的每个单词，我们都有可能将其有效的从文本中删除。删除单词的概率与单词的频率有关。

如果我们的窗口大小为 10，并且从文本中删除“the”的特定实例：

- 1 当我们训练剩余的单词时，“the”不会出现在任何上下文窗口中。  

- 2 当“the”是输入词时，我们将减少 10 个训练样本。

请注意这两个效果如何帮助解决上述两个问题。

## Sampling rate  

word2vec C 代码实现了计算词汇表中给定单词的概率的等式。

wi 是单词，z（wi）是语料库中所有单词的分数。例如，如果单词“peanut”在 10 亿字的语料库中出现 1000 次，则 z（'peanut'）= 1E-6。

代码中还有一个 'sample' 参数，用于控制发生多少次采样，默认值为 0.001。较小的“sample”意味着 words 更难以保留。

P（wi）是保留单词的概率：

$P(w_i) = (\sqrt {\frac {z(w_i)}{0.001}} + 1) \cdot \frac {0.001}{z(w_i)}$

您可以在Google中快速绘制此图形以查看形状。  

没有一个单词应该是语料库中非常大的比例，所以我们希望在 x 轴上查看相当小的值。

这里有一些有趣的地方在这个函数（也是使用默认的样本值0.001）。

- 当 z（wi）≤ 0.0026 时，P（wi）= 1.0（保持100%的概率）。这意味着只有占总词数超过 0.26% 的词将被二次抽样。  

- 当 z（wi）= 0.00746 时，P（wi）= 0.5（保持50%的机会）。  

- 当z（wi）= 1.0，P（wi）= 0.033（保持3.3%的机会）。也就是说，如果语料库完全由 wi 词组成，这当然是荒谬的。  

你可能会注意到这篇论文定义的这个函数与 C 代码中实现的有些不同，但我认为 C 实现是更权威的版本。  

## Negative Sampling  

训练一个神经网络意味着输入一个训练样本就稍微调整所有的神经元权重，以便更准确地预测训练样本。换句话说，每个训练样本都会调整神经网络中的所有权重。

正如我们上面所讨论的那样，我们的单词词汇的大小意味着我们的 skip-gram 神经网络有很多权重，所有这些都将被我们数十亿训练样本中的每一个稍微更新！

负采样通过让每个训练样本只修改一小部分权重而不是全部权重来解决这个问题。这是它的工作原理。

当在单词对(“fox”, “quick”)上训练网络时，请回想一下，网络的“标签”或“正确输出”是一个 one-hot 向量。也就是说，对于对应于“quick”的输出神经元输出 1，并且对于所有其他数千个输出神经元输出 0。

对于 negative  抽样，我们将随机选择一小部分“negative ”单词（比如 5）来更新权重。（在这种情况下，“negative ”一词是我们希望网络输出 0 的那个词）。我们还将更新我们的“positive”单词的权重（在我们当前的例子中，这是“quick”一词）。

该论文指出，选择 5-20 个单词适用于较小的数据集，并且对于大型数据集只能使用 2-5 个单词。  

回想一下，我们模型的输出层有一个 300 x 10,000 的权重矩阵。所以我们只是更新我们正面词的权重（“quick”），加上我们想要输出的其他 5 个词的权重。这总共有 6 个输出神经元，总共有 1,800 个权重值。这只是输出层 3M weights 的 0.06%！

在隐藏层中，只更新输入词的权重（无论您是否使用 Negative 采样，情况都是如此）。

## Selecting Negative Samples  

“负样本”（即我们将训练输出为 0 的 5 个输出字）是使用“unigram distribution”选择的。

本质上，选择一个词作为负样本的概率与其频率有关，更频繁的词更可能被选作负样本。

在 word2vec C 实现中，您可以看到这个概率的等式。每个单词的权重等于其频率（字频）的 3/4 次方。选择一个词的概率只是它的权重除以所有单词的权重之和。

$P(w_i) = \frac {f(w_i)^{3/4}}{\sum_{j=0}^{n}(f(w_i)^{3/4})}$

将频率提高到 3/4 power 的决定似乎是经验性的；在他们的论文中他们说它超越了其他数字。您可以查看函数的形状 - 只需在Google中输入：“plot y = x^(3/4) and y = x”，然后放大范围 x = [0，1]。它有一个略微增加的曲线。

这种选择在 C 代码中实现的方式很有趣。他们有一个拥有 100M 元素的大array （他们称之为 unigram table）。他们多次使用用词汇表中每个单词的索引填充这个表格，单词索引出现在表格中的次数由 $P(w_i) * table_size$ 给出。然后，为了实际选择一个负样本，您只需生成一个介于 0 到 100M 之间的随机整数，然后在表格中使用该索引处的单词。由于较高的概率词在表格中出现次数更多，因此您更有可能选择这些词语。  

# Enriching Word Vectors with Subword Information  

## Abstract  

在大型未标记语料库上训练的连续词表示对许多自然语言处理任务都很有用。学习这种表示的流行模型通过为每个单词分配一个不同的矢量来忽略单词的形态。这是一个限制，特别是对于词汇量很大且词汇很多的语言。在本文中，我们提出了一种基于 skip-gram 模型的新方法，其中每个单词表示为a
bag of character n-grams。矢量表示与每个字符 n-gram 相关联；单词被表示为这些表示的总和。我们的方法速度很快，可以快速训练大型语料库中的模型，并允许我们计算未出现在训练数据中的单词的单词表示。我们用九种不同的语言来评估我们的词汇表达，包括词汇相似性和类比任务。通过与最近提出的形态词表示进行比较，我们证明了我们的向量在这些任务上达到了最先进的性能。  

## 1 Introduction  

学习连续的词汇表达在自然语言处理方面有着悠久的历史。这些表示通常从大型未标记语料中使用共现统计计算出来。称为分布式语义学的大量工作，研究了这些方法的性质。在神经网络社区中，Collobert 和 Weston（2008）提出使用前馈神经网络学习单词嵌入，通过基于左侧两个单词和右侧两个单词预测单词。最近，米科洛夫等人。提出了简单的对数双线性模型来有效地学习非常大的语料库上的词的连续表示。  

这些技术中的大多数通过不同的向量表示词汇表中的每个词，而不用参数共享。尤其是，他们忽略了单词的内部结构，这是形态丰富的语言（如土耳其语或芬兰语）的重要限制。例如，在法语或西班牙语中，大多数动词有超过四十种不同的变形形式，而芬兰语则有十五种名词形式。这些语言包含许多在训练语料库中很少发生（或根本不发生）的单词形式，这使得难以学习好的单词表示。由于许多单词遵循规则形成，因此可以通过使用字符级别信息来改进形态丰富的语言的向量表示。  

在本文中，我们建议学习字符 n-gram 的表示，并将单词表示为 n-gram 向量的总和。我们的主要贡献是引入连续 skip-gram 模型的扩展，其中考虑了子字词信息。我们用九种不同形态的语言评估这个模型，显示出我们方法的好处。  

## 2 Related work  

**Morphological word representations** 近年来，已经提出了许多方法将形态信息结合到词表示中。为了更好地模拟稀有词汇，Alexandrescu 和Kirchhoff（2006）引入了分解神经语言模型，其中词语表示为特征集合。这些特征可能包括形态信息，并且该技术已成功应用于形态丰富的语言，如土耳其语（Sak等，2010）。最近，一些作品提出了不同的组合函数来从语素中推导词的表示。这些不同的方法依赖于词的形态分解，而我们的则不是。同样，Chen 等人介绍了一种联合学习汉字和字符嵌入的方法。Cui 等人提出将形态上相似的词限制为具有相似的表示。Soricut 和 Och（2015）描述了一种学习形态变换的向量表示的方法，允许通过应用这些规则来获取未看见的单词的表示。Cotterell 和 Schütze（2015年）介绍了形态学注释数据的词汇表征。最接近我们的方法，Schütze（1993）通过奇异值分解学习了 four-grams 字符的表示，并且通过对 four-grams 表示求和而得出单词的表示。最近，Wieting等人（2016）也提出使用字符 n-gram 计数向量表示单词。然而，用于学习这些表示的目标函数基于释义对，而我们的模型可以在任何文本语料库上进行训练。  

**Character level features for NLP** 与我们的工作密切相关的另一个研究领域是自然语言处理的字符级模型。这些模型将单词分割成字符并旨在直接从字符学习语言表示。第一类这样的模型是递归神经网络，应用于语言建模，文本规范化，词性标注和解析。另一个模型家族是卷积神经网络训练的字符，应用于词性标注，情感分析，文本分类和语言建模。Sperr 等人引入了一种基于受限玻尔兹曼机器的语言模型，其中词语被编码为一组字符节点。最后，最近在机器翻译方面的工作已经提出使用子字词单元来获得罕见字词的表示。  

## 3 Model  

在本节中，我们建议我们的模型在考虑形态学的同时学习单词表示。我们通过考虑子字单位来对形态进行建模，并且用字符 n-gram 的总和来表示字。我们将首先介绍我们用来训练单词向量的一般框架，然后展示我们的子词模型并最终描述我们如何处理字符 n-gram 的词典。  

### 3.1 General model  

我们首先简要回顾一下 Mikolov 等人提出的 skip-gram 模型。我们的模型来源于此。给定一个大小为 W 的单词词汇表，其中一个单词由其索引 $w \in \{1,...,W\}$ 来标识，目标是学习每个单词 w 的向量表示。受分布假设（Harris，1954）的启发，词汇表征被训练以预测出现在其上下文中的词汇。更正式地说，给定一个大的训练语料库，skip-gram 模型的目标是最大化以下对数似然函数：  

$\sum_{t=1}^{T} \sum_{c \in C_t} \log {p(w_c|w+t)}$

其中上下文 Ct 是围绕单词 wt 的词的索引的集合。使用上述单词向量来参数化给定 wt 的上下文词 wc 的概率。现在，让我们考虑一下，我们得到了一个评分函数 s，它将 (word, context)对映射为分数。  

定义上下文词的概率的一个可能选择是 softmax：  

$p(w_c|w_t) = \frac {e^{s(w_t,w_c)}}{\sum_{j=1}^W e^{s(w_t,j)}}$

然而，这样的模型并不适用于我们的情况，因为它暗示，给定一个单词 wt，我们只能预测一个上下文单词 wc。  

预测上下文单词的问题可以被定义为一组独立的二进制分类任务。然后，目标是独立预测上下文单词的存在（或不存在）。对于位置 t 处的单词，我们将所有上下文单词视为正例，并从字典中随机 sample negatives。对于选择的上下文位置 c，使用二元逻辑损失，我们获得以下负对数似然性：  

$\log{(1+e^{-s(w_t,w_c)})} + \sum_{n \in N_{t,c} \log {(1+e^{s(w_t,n)})}}$

其中 $N_{t,c}$ 是从词汇中抽取的一组反例。通过表示逻辑损失函数：$l: x \rightarrow \log{1+e^{-x}}$，我们可以将目标重新写为：  

$\sum_{t=1}^T \left[ \sum_{c \in C_t} l(s(w_t,w_c)) + \sum_{n \in N_{t,c}} l(-s(w_t,n)) \right]$

词 wt 和上下文词 wc 之间的评分函数使用词向量参数化。让我们为词汇中的每个单词 w 定义 $R^d$ 中的两个向量 $u_w$ 和 $v_w$。这两个向量在文献中有时被称为输入和输出向量。特别是，我们有向量 $u_{wt}$ 和 $v_{wc}$，分别对应于 wt 和 wc。然后可以将单词和上下文向量之间的标量积作为得分。本节描述的模型是 Mikolov 等人提出的负样本抽样模型。  

### 3.2 Subword model  

通过为每个单词使用不同的矢量表示形式，skip-gram 模型将忽略单词的内部结构。在本节中，我们提出了一个不同的评分函数 s，以便考虑到这些信息。  

每个单词 w 表示为 a bag of character n-gram。我们在单词的开头和结尾添加特殊的边界符号 $<$ 和 $>$，以便从其他字符序列中区分前缀和后缀。 我们还将 w 本身包含在它的 n 元组中，以学习每个单词的表示（in addition to character n-grams）。以 where 和 n = 3 这个词为例，它将由字母 n-grams 表示：  

$<wh, whe, her, ere, re>$

以及特殊的序列  

$<where>$

请注意，对应于 $<her>$ 这个词的序列不同于 where 中的 tri-gram her。在实践中，我们提取 n 大于或等于3，小于或等于 6 的所有 n 元组。这是一个非常简单的方法，可以考虑不同的 n 元组，例如采用所有的前缀和后缀。  

假设给你一个大小为 G 的 ngram 字典，给定一个单词 w，让我们用 $g_w \subset \{1,...,G\}$ 表示出现在 w 中的一组 n-gram。我们对 n-gram 中每个 g 都分配一个 $z_g$。通过其 n-gram 的向量表示的总和来表示一个词。我们因此获得评分函数：  

$s(w,c) = \sum_{g \in g_w} z^T_g v_c$

这个简单的模型允许跨越单词共享表示，从而允许学习罕见单词的可靠表示。  

为了限制我们模型的内存需求，我们使用哈希函数将 n-gram 映射为 1 到 K 中的整数。我们使用 Fowler-Noll-Vo 哈希函数。我们设置 $K = 2.10^6$以下。最终，一个单词由单词字典中的索引以及它包含的一组散列 n-gram 来表示。  

## 4 Experimental setup  

### 4.1 Baseline  

在大多数实验中，我们将模型与 word2vec2 包中 skip-gram 和 cbow 模型的 C 实现进行比较。  

### 4.2 Optimization  

我们通过对之前呈现的负对数似然性执行随机梯度下降来解决我们的优化问题。所有线程以异步方式共享参数和更新向量。  

### 4.3 Implementation details  

对于我们的模型和基线实验，我们使用以下参数：单词向量的维数为 300。对于每个正例，我们随机抽样 5 个负样本。我们使用大小为 c 的上下文窗口，并在 1 到 5 之间对大小 c 进行均匀采样。为了对最频繁的单词进行二次采样，我们使用 $10^{-4}$ 的拒绝阈值。在建立单词词典时，我们保留在训练集中出现至少五次的单词。对于 skip-gram 基线，步长 $\gamma_0$ 设置为 0.025，对于我们的模型和 cbow 基线，步长 $\gamma_0$ 设置为 0.05。这些是 word2vec 软件包中的默认值，也适用于我们的模型。  

在英文数据上使用此设置，我们的字符 n-gram 模型比 skip-gram 基线的训练慢大约 1.5 倍。事实上，我们处理 105k words/second/thread vs  基线的145k words/second/thread。我们的模型是用 C++ 实现的，并且是公开可用的。  

### 4.4 Datasets  

除了与以前的工作（5.3节）相比较外，我们还对维基百科数据进行了模型训练。我们用阿拉伯语，捷克语，德语，英语，西班牙语，法语，意大利语，罗马尼亚语和俄语下载了九种语言的维基百科转储。我们使用 Matt Mahoney 的预处理 perl 脚本对原始维基百科数据进行了规范化。所有数据集都进行了混洗，并且我们通过对它们进行了五遍训练来模拟我们的模型。  

## 5 Results  

我们在五个实验中评估我们的模型：评估单词相似性和单词类比，与最先进的方法进行比较，分析训练数据大小以及字符 n-gram 的大小的影响。我们将在下面的章节中详细介绍这些实验。  

### 5.1 Human similarity judgement  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/1.jpg?raw=true)

我们首先评估我们的表示在词语相似性/相关性任务上的质量。我们通过计算 Spearman 的等级相关系数来判断人类判断与矢量表示之间的余弦相似度。对于德语，我们比较了三个数据集上的不同模型：GUR65，GUR350 和 ZG222。对于英语，我们使用 Finkelstein 等人介绍的 WS353 数据集，以及 Luong 等人提出的稀有词数据集（RW）。我们评估翻译数据集 RG65 上的法语单词向量。使用（Hassan和Mihalcea，2009）中描述的数据集评估西班牙语，阿拉伯语和罗马尼亚语单词向量。使用 Panchenko 等人介绍的 HJ 数据集评估俄语单词向量。  

我们在表1 报告了我们的方法和基线方法在所有数据集的结果。这些数据集中的一些单词没有出现在我们的训练数据中，因此我们无法使用 cbow 和skip-gram 基线获取这些单词的单词表示。为了提供可比较的结果，我们建议默认使用这些词的空向量。由于我们的模型利用了子词信息，因此我们还可以计算词外单词的有效表示。我们通过取其 n-gram 向量的总和来做到这一点。当使用空向量表示 OOV 词时，我们将我们的方法称为 sisg- 和 sisg。  

首先，通过查看表1，我们注意到使用子字信息的所提出的模型（sisg）优于除了英语 WS353 数据集之外的所有数据集的基线。此外，词汇外单词（sisg）的计算向量总是至少与不这样做（sisg-）一样好。这证明了以字符 n-gram 形式使用子字信息的优点。  

其次，我们观察到使用字符 n-gram 对阿拉伯语，德语和俄语的影响比英语，法语或西班牙语更重要。德语和俄语展示语法变体，其中德语四种，俄语六种。另外，很多德语单词都是复合词; 例如，名词短语“table tennis”用一个词语写成“Tischtennis”。通过利用“Tischtennis”和“Tennis”之间的字符级相似性，我们的模型不会将这两个单词表示为完全不同的单词。  

最后，我们观察到在 English Rare Words 数据集（RW）上，对于 OOV 词我们的方法优于基线。这是因为英语 WS353 数据集中的单词是不使用子字词信息就可以获得良好矢量的常用单词。当评估不太常见的单词时，我们发现在单词之间的字符级别使用相似性可以帮助学习良好的单词向量。  

### 5.2 Word analogy tasks  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/2.jpg?raw=true)

我们现在评估我们的词类比问题的方法，形式是 A is to B as C is to D，其中 D 必须由模型预测。我们使用 Mikolov 等人介绍的数据集。分别有捷克语，英语，德语，意大利语。有些问题包含了我们训练语料库中没有出现的词汇，因此我们从评估中排除了这些问题。  

我们在表2 中报告了不同模型的准确性。我们观察到形态信息显着改善了语法任务；我们的方法胜过基线。相反，它不利于语义问题，甚至会降低德语和意大利语的表现。请注意，这与我们考虑的字符 n-gram 长度的选择密切相关。我们在 5.5 节展示，当 n-gram 的大小被选择为最佳时，语义类比降低得更少。另一个有趣的发现是，正如预期的那样，对形态丰富的语言如捷克语和德语改进的更多。  

### 5.3 Comparison with morphological representations  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/3.jpg?raw=true)

我们还将我们的方法与以前关于词相似任务的工作进行比较。使用的方法是：Luong 等人的递归神经网络，Qiu 等人的 morpheme cbow，以及 Soricut 和Och 的形态转换。为了使结果具有可比性，我们使用与我们比较的方法相同的数据集对我们的模型进行了训练。我们还将我们的方法与 Botha 和 Blunsom 引入的对数双线性语言模型进行了比较，该模型在 Europarl 和新闻评论语料库上进行了训练。再次，我们使用相同的数据对我们的模型进行了训练，以使结果可比。使用我们的模型，我们通过对字符 n-gram 的表示进行求和来获得词外词的表示。我们在表3 中报告了结果。我们观察到，与基于从形态 segmentors 获得的子词信息的技术相比，我们的简单方法表现良好。我们还观察到，我们的方法优于基于前缀和后缀分析的 Soricut 和 Och 的方法。德语的巨大改进是由于他们的方法不能模拟名词复合，与我们的相反。  

### 5.4 Effect of the size of the training data  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/4.jpg?raw=true)

由于我们利用单词之间的字符级相似性，我们能够更好地模拟不常用的单词。因此，我们也应该对我们使用的训练数据的大小更加健壮。为了评估这一点，我们建议训练集大小作为函数来评估我们的词向量在相似任务上的效果。我们在图1 中报告结果。  

正如在 5.1 节中介绍的实验一样，并非评估集的所有单词都出现在维基百科数据中。同样，默认情况下，我们对这些单词使用空矢量（sisg-）或通过对 n-gram 表示（sisg）进行求和来计算矢量。随着数据集的收缩，词外率越来越高，因此 sisg- 和 cbow的性能必然下降。然而，所提出的模型（sisg）将 non-trivial 的向量分配给以前未见过的单词。  

首先，我们注意到对于所有数据集和所有大小，所提议的方法（sisg）表现比基线好。然而，随着越来越多的数据可用，基线 cbow 模型的性能会变得更好。另一方面，我们的模型似乎很快饱和，增加更多的数据并不总是会导致改进的结果。  

其次，最重要的是，我们注意到，即使使用非常小的训练数据集，所提出的方法也提供了非常好的单词向量。例如，在德国的 GUR350 数据集上，我们的模型（sisg）训练了 5% 的数据比完整数据集训练的 cbow 基线获得了更好的性能。另一方面，在英国 RW 数据集上，使用维基百科语料库的 1%，我们得到45 的相关系数，这比在整个数据集上训练的 cbow 的性能要好。这具有非常重要的实际意义：可以在有限大小的数据集上计算性能良好的单词向量，并且仍能很好地处理以前未见过的单词。通常，在特定应用程序中使用 vectorial 字表示时，建议对与应用程序相关的文本数据重新训练模型。  

### 5.5 Effect of the size of n-grams  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/5.jpg?raw=true)

所提出的模型依赖于使用字符 ngram 来表示单词作为向量。正如在 3.2 节中提到的。我们决定使用 3 到 6 个字符的 n-gram。这种选择是任意的，因为这些长度的 n-gram 将涵盖范围广泛的信息。它们将包括短后缀以及更长的根。在这个实验中，我们凭经验检查了 n-gram范围对性能的影响。我们在表4 中报告英语和德语关于词相似性和类比数据集的结果。  

我们观察到，对于英语和德语，我们任意选择 3-6 是一个合理的决定，因为它提供了令人满意的跨语言表现。长度范围的最佳选择取决于所考虑的任务和语言，应适当调整。但是，由于测试数据的稀缺性，我们没有实施任何适当的验证程序来自动选择最佳参数。尽管如此，如 3-6 等较大范围提供了合理数量的子词信息。  

该实验还表明，包括长 n-gram 是很重要的。对于德语来说尤其如此，因为许多名词都是由几个单位组成的 compounds，只能通过较长的字符序列才能被捕获。在类比任务上，我们观察到使用更大的 n-gram 有助于语义类比。然而，结果总是通过取 n≥3 而不是 n≥2 来改善，这表明字符 2-gram 对于该任务没有提供信息。如3.2节所述，在计算字符 n-gram 之前，我们添加和附加特殊的位置字符来表示字的开始和结束。  

### 5.6 Language modeling  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/6.jpg?raw=true)

在本节中，我们描述了在语言建模任务中使用我们的方法获得的单词向量的评估。我们使用 Botha 和 Blunsom（2014）介绍的数据集，以五种语言（CS，DE，ES，FR，RU）评估我们的语言模型。每个数据集大约包含一百万个训练 tokens，我们使用与 Botha 和 Blunsom（2014）相同的预处理和数据拆分。  

我们的模型是一个具有 650 个 LSTM 单元的递归神经网络，regularized with dropout（概率为0.5）和 weight decay（正则化参数为10-5）。我们使用Adagrad 算法学习参数，学习率为 0.1，剪切大于1.0范数的梯度。 我们在 [-0.05,0.05] 范围内初始化网络的权重，并使用 20 的批处理大小。考虑两个基线：我们将我们的方法与 Botha 和 Blunsom（2014）的对数双线性语言模型以及 Kim 等人的字符感知语言模型相比。我们在语言建模任务的训练集上训练了带有字符 n-gram 的单词向量，并使用它们来初始化我们的语言模型的查找表。我们报告我们的模型的测试困惑，而不使用预先训练的单词向量（LSTM），预先训练没有子词信息（sg）和我们的向量（sisg）的单词向量。结果列于表5。  

我们观察到，使用预先训练的词表示初始化语言模型的查找表可以提高基线 LSTM 上的测试困惑度。最重要的观察结果是使用用子字信息训练的字表示胜过普通的 skip-gram 模型。  

## 6 Qualitative analysis  

### 6.1 Nearest neighbors  

我们在表7 中报告样本定性结果。对于选定的单词，我们根据余弦相似性显示使用所提议方法训练的矢量和 skipgram 基线的最近邻居。正如预期的那样，使用我们的方法的复杂，技术和不常用词语的最近邻居比使用基线模型获得的更好。  

### 6.2 Character n-grams and morphemes  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/4/7.jpg?raw=true)

我们想定性评估一个单词中最重要的 n 元组是否对应于 morphemes。为此，我们将一个单词向量构造成 n-gram 的总和。如 3.2 节所述，每个单词 w 表示为它的 n 元组的总和。  

我们在表6 中以三种语言显示选定词的 ranked n-gram。  

对于有很多复合名词的德语，我们观察到最重要的 n-gram 对应于有效的词素。好例子包括 Autofahrer（汽车司机），其中最重要的 n-gram 是Auto（汽车）和 Fahrer（司机）。  

### 6.3 Word similarity for OOV words  

如 3.2 节所述，我们的模型能够为未出现在训练集中的单词构建单词向量。对于这样的词，我们简单地平均它的 n 元组的向量表示。为了评估这些表示的质量，我们通过从英语 RW 相似性数据集中选择几个单词对来分析哪些 n-gram 匹配最适合OOV单词。对于每一对单词，我们显示出现在单词中的每对 n-gram 之间的余弦相似度。为了模拟具有更多 OOV 字的设置，我们使用在维基百科数据的 1% 上训练的模型。结果如图2 所示。  

我们观察有趣的模式，显示子词匹配正确。事实上，对于 chip 这个词，我们清楚地看到在 microcircuit 中有两组 n-gram 匹配得很好。这些大致对应于micro 和 circuit，并且中间的 n-gram 不匹配。  

### 7 Conclusion  

在本文中，我们调查了一种简单的方法，通过考虑子词信息来学习单词表示。我们的方法将字符 n-gram 结合到 skip-gram 模型中。由于其简单性，我们的模型训练速度快，不需要任何预处理或监督。我们表明，我们的模型胜过了不考虑子词信息的基线，以及依赖于形态分析的方法。我们将开源实现我们的模型，以便于比较未来在学习子词表示方面的工作。

# Bag of Tricks for Efficient Text Classification  

## Abstract  

本文探讨了一种简单有效的文本分类基准。实验表明，我们的快速文本分类 fastText 在准确性方面通常与深度学习分类器相当，在训练和评估方面速度快很多。我们可以在不到 10 分钟的时间内使用标准的多核 CPU 对超过 10 亿个单词进行快速文本训练，并在不到一分钟的时间内对 312K 类中的 50 万个句子进行分类。  

## 1 Introduction  

文本分类是自然语言处理中的一项重要任务，具有许多应用，如网络搜索，信息检索，排序和文档分类。最近，基于神经网络的模型变得越来越流行。虽然这些模型在实践中取得了非常好的表现，但是它们在 train 和 test 时往往相对较慢，限制了它们在非常大的数据集上的使用。  

同时，线性分类器通常被认为是文本分类问题的强基线。尽管它们很简单，但如果使用了正确的特征，他们通常会获得最先进的性能。他们也有可能扩展到非常大的语料库。  

在这项工作中，我们探讨了如何在文本分类的背景下将这些基线扩展为具有大输出空间的非常大的语料库。受近期有效词表征学习的启发，我们表明具有rank 约束和快速损失近似的线性模型可以在十分钟内训练十亿字，同时实现与最先进的技术相媲美的性能。我们在两项不同任务中评估我们的方法 fastText 的质量，即标签预测和情感分析。  

## 2 Model architecture  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/5/1.jpg?raw=true)

句子分类的简单而有效的基线是将句子表示为词袋（BoW）并且训练线性分类器，例如逻辑回归或 SVM。但是，线性分类器不会在特征和类之间共享参数。 这可能会限制它们在大输出空间的情况下的泛化，其中一些类只有很少的例子。这个问题的常见解决方案是将线性分类器分解成低秩矩阵或使用多层神经网络。  

图1 显示了一个 rank 约束的简单线性模型。第一个权重矩阵 A 是单词上的查找表。然后将单词表示平均为文本表示，然后将其馈送给线性分类器。文本表示是一个可能被重用的隐藏变量。这种结构类似于 Mikolov 等人的 cbow 模型，其中中间词被标签取代。我们使用 softmax 函数 f 来计算预定义类中的概率分布。对于一组 N 个文档，这导致了对这些类的负对数似然性的最小化。  

### 2.1 Hierarchical softmax  

当类的数量很大时，计算线性分类器的计算量很大。更确切地说，计算复杂度为O(kh)，其中 k 是类的数量，h 是文本表示的维数。为了改善我们的运行时间，我们使用基于霍夫曼编码树的分层 softmax。在训练期间，计算复杂度降至 O(h log2(k))。  

当搜索最可能的类别时，分层 softmax 在测试时间也是有利的。每个节点都与从根节点到该节点的路径概率相关联。节点的概率总是低于其父节点的概率。通过深度探索树首先搜索并跟踪叶子之间的最大概率使我们能够丢弃与小概率相关的任何分支。在实践中，我们观察到在测试时间O(h log2(k))的复杂度降低。这种方法进一步扩展到以 O(log)T))为代价，使用 binary heap 计算 T-top 目标。  

### 2.2 N-gram features  
词袋是单词顺序不变的，但考虑到这个顺序通常在计算上非常昂贵。相反，我们 a bag of n-grams 作为附加特征来捕获有关词序的部分信息。这在实践中是非常有效的，同时对明确使用顺序的方法实现可比较的结果。  

我们通过使用 hashing 技巧，使用与 Mikolov 等人相同的 hashing 函数，保持了快速和高效的 n-gram 映射。如果我们只使用 bigrams 就需要 10M
bins，否则 100M。  

## 3 Experiments  

我们在两个不同的任务上评估 fastText。首先，我们将它与情感分析问题上的现有文本分类进行比较。然后，我们评估其扩展到标签预测数据集上的大输出空间的能力。请注意，我们的模型可以用 Vowpal Wabbit 库实现，但我们在实践中观察到，我们的定制实现速度至少提高了2-5倍。  

### 3.1 Sentiment analysis  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/5/2.jpg?raw=true)

**Datasets and baselines** 我们采用 Zhang 等人的 8 个数据集和评估协议。我们报告 Zhang 等人的 n-gram 和 TFIDF 基线，以及 Zhang 和LeCun的字符级卷积模型（char-CNN），Xiao 和 Cho 的基于字符的卷积循环网络（char-CRNN）和非常深的卷积网络 （VDCNN）等。  

**Results** 我们在图1 中给出了结果。我们使用 10 个隐藏单元并运行 5 个 epoch 的 fastText，并在 {0.05,0.1,0.25,0.5} 的验证集上选择了一个学习率。在这个任务中，添加 bigram 信息将使性能提高 1-4%。总的来说，我们的准确率比 char-CNN 和 char-CRNN 略好，并且比 VDCNN 差一些。请注意，我们可以通过使用更多的 n-gram 来略微提高精度，例如使用 trigrams，搜狗数据的性能可以达到 97.1%。最后，图3 显示我们的方法与 Tang 等人提出的方法具有可比性。我们调整验证集上的超参数，并观察使用高达 5 的 n-grams 可以获得最佳性能。不像 Tang 等人，fastText 不使用预先训练的词嵌入，这可以解释 1% 的准确率差异。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/5/3.jpg?raw=true)

**Training time** char-CNN 和 VDCNN 都使用 NVIDIA Tesla K40 GPU 进行训练，而我们的模型则使用 20 个线程在 CPU 上进行训练。表2 显示使用卷积的方法比 fastText 慢几个数量级。虽然使用更新的 CUDA 实现的卷积可以使 char-CNN 的速度提高 10 倍，但 fastText 只需不到一分钟的时间就可以训练这些数据集。Tang 等人的 GRNN 方法，每个 CPU 使用一个线程大约需要 12 个小时。与基于神经网络的方法相比，我们的加速比随着数据集的大小而增加，至少达到 15,000 倍的加速。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/5/4.jpg?raw=true)

### 3.2 Tag prediction  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/5/5.jpg?raw=true)

**Dataset and baselines** 为了测试我们方法的可伸缩性，我们对 YFCC100M 数据集进行了进一步评估，该数据集由几乎 100M 的带有标题，标题和标签的图像组成。我们专注于根据标题预测标签（我们不使用图像）。我们删除少于 100 次的字词和标签，并将数据分成 train,，验证和测试集。train 包含 91,188,648 个示例（1.5B tokens）。验证集有 930,497 个示例和测试集 543,424。词汇大小为 297,141，并且有 312,116 个独特标签。我们将发布一个脚本来重新创建这个数据集，以便我们的数据可以被复制。我们在1 报告精度。  

我们考虑基于频率的基线预测最频繁的标签。我们还与 Tagspace 进行了比较，这是一个类似于我们的标签预测模型，但是基于 Weston 等人的 Wsabie 模型，虽然使用卷积描述标签空间模型，但我们认为线性版本可以实现可比较的性能，但速度更快。  

**Results and training time** 表5 给出了 fastText 和基线的比较。我们运行 5 个 epoch 的 fastText，并将其与 Tagspace 的两种尺寸的隐藏层（即50和200）进行比较。两个模型都实现了与隐藏层相似的性能，但添加 bigrams 可以显着提高精度。在测试时，Tagspace 需要计算所有类别的分数，这使得它相对较慢，而当类别数量很多（此处超过300K）时，我们的快速推理会显著提高速度。总体而言，获得质量更好的模型的速度要快一个数量级。 测试阶段的加速甚至更加显着（600倍加速）。表4 显示了一些定性的例子。  

## 4 Discussion and conclusion  

在这项工作中，我们提出了一种简单的文本分类基线方法。与来自 word2vec 的无监督训练的单词向量不同，我们的单词特征可以被平均在一起以形成好的句子表示。在几个任务中，fastText 获得的性能与最近提出的深度学习方法相当，而且速度更快。尽管深层神经网络在理论上比浅层模型具有更高的表征能力，但是如何简单的文本分类问题（例如情感分析）来评估它们是不错的。我们将发布我们的代码，以便研究团体可以轻松构建我们的工作。

# Massive Exploration of Neural Machine Translation Architectures  

## Abstract  

过去几年神经机器翻译（NMT）已经显示出显著的进步，现在正在将生产系统部署到最终用户。当前体系结构的一个主要缺点是它们的训练成本很高，通常需要几天到几周的 GPU 时间才能收敛。这使得穷举的超参数搜索（通常与其他神经网络架构一起完成）非常昂贵。在这项工作中，我们首先对 NMT 体系结构超参数进行大规模分析。我们报告了数百次实验运行的实验结果和方差数字，对应于标准 WMT 英语至德语翻译任务超过 250,000 个GPU小时。我们的实验为构建和扩展 NMT 体系结构提供了新颖的见解和实用建议。作为这项贡献的一部分，我们发布了一个开放源代码的 NMT 框架，使研究人员能够轻松实验新技术并重现最先进的结果。  

## 1 Introduction  

神经机器翻译（NMT）（Kalchbrenner 和 Blunsom，2013; Sutskever 等，2014; Cho等，2014）是自动翻译的端到端方法。NMT 已经取得了令人瞩目的成果（Jean et al，2015; Luong et al，2015b; Sennrich et al，2016a; Wu et al，2016），超越了基于短语的系统，同时解决了诸如需要手工特征等缺点。NMT 最流行的方法是基于由两个递归神经网络（RNN）组成的编码器-解码器体系结构以及将目标与源标记对齐的注意机制（Bahdanau 等，2015; Luong等，2015a）。  

目前的 NMT 体系结构的一个缺点是训练它们所需的计算量。对数百万个示例的实际数据集进行训练通常需要数十个 GPU，并且收敛时间大约为几天到几周。虽然扫描大型超参数空间在计算机视觉中很常见（Huang et al，2016b），但这种探索对于 NMT 模型来说过于昂贵，将研究人员限制在完善的架构和超参数选择上。此外，还没有关于架构超参数如何影响 NMT 系统性能的大规模研究。因此，目前仍不清楚为什么这些模型的表现与他们一样好，以及我们如何改进。  

在这项工作中，我们首次对神经机器翻译系统的架构超参数进行综合分析。我们总共使用超过 250,000 小时的 GPU 时间，探索 NMT 架构的常见变体，并深入了解哪些架构选择最重要。我们报告所有实验的 BLEU 分数，困惑度，模型大小和收敛时间，包括在每个实验的几次运行中计算的方差数。另外，我们向公众发布了一个用于运行实验的新软件框架。  

总之，这项工作的主要贡献如下：  

- 我们提供有关神经机器翻译模型优化的有用见解，并为未来的研究提供有希望的方向。例如，我们发现深度编码器比解码器更难以优化，dense 残差连接比常规残差连接产生更好的性能，LSTM 优于 GRU，并且良好调整的 beam search 对于获得最先进的结果至关重要。通过为选择基线架构提供实用建议，我们帮助研究人员避免在未经验证的模型变化上浪费时间。  

- 我们还确定了 BLEU 等指标受随机初始化和轻微超参数变化影响的程度，帮助研究人员将统计上显著的结果与随机噪声区分开来。  

- 最后，我们发布基于 TensorFlow 的开放源代码软件包，专门设计用于实现可重现的序列-序列模型。所有的实验都是使用这个框架进行的，我们希望通过向公众公布加速未来的研究。我们还发布了本文中重现实验所需的所有配置文件和处理脚本。  

## 2 Background and Preliminaries  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/1.jpg?raw=true)

### 2.1 Neural Machine Translation  

我们的模型基于具有注意力机制的 encoder-decoder 架构（Bahdanau等，2015; Luong等，2015a），如图1 所示。编码器函数 $f_{enc}$ 将源 tokens 序列 $x = (x_1,...,x_m)$ 作为输入，并产生一系列状态 $h = (h_1,...,h_m)$。 在我们的基础模型中，$f_{enc}$ 是一个双向的 RNN，而状态 $h_i$ 对应于由后向和前向 RNN 产生的状态的拼接，$h_i = [\overrightarrow h_i; \overleftarrow h_i]$。解码器 $f_{dec}$ 是基于 h 预测目标序列 $y = (y_1,...,y_k)$ 概率的 RNN。基于解码器 RNN 中的循环状态 $s_i$，前一个词 $y_{<i}$ 和上下文向量 $c_i$ 来预测每个目标 token $y_i \in 1,..,V$ 的概率。上下文向量 $c_i$ 也被称为注意力向量，并被计算为源状态的加权平均值。  

$c_i = \sum_j a_{i,j}h_j \tag{1}$
$a_{i,j} = \frac {\hat a_{i,j}}{\sum_j \hat a_{i,j}} \tag{2}$
$\hat a_{i,j} = att(s_i,h_i) \tag{3}$

这里，$att(s_i,h_i)$ 是计算编码器状态 $h_j$ 与解码器状态 $s_i$ 之间的非标准化对准分数的注意力函数。在我们的基本模型中，我们使用形式为 $att(s_i,h_i) = \left\langle W_h h_j,W_s s_i \right\rangle$ 的函数，其中矩阵 W 用于将 source and target states 转换为相同大小的表示。  

解码器在固定大小 V 的词汇表上输出分布：  

$P(y_i|y_1,...,y_{i-1},x) = softmax(W[s_i；c_i] + b)$

通过使用随机梯度下降来最小化目标词的负对数似然性，整个模型被端对端地训练。  

## 3 Experimental Setup  

### 3.1 Datasets and Preprocessing  

我们通过结合 Europarl v7，新闻评论 v10 和 Common Crawl 语料库组成 4.5M 句子，在对 WMT’15 English→German 任务进行所有实验。我们使用newstest2013 作为我们的验证集，newstest2014 和 newstest2015 作为我们的测试集。为了检验通用性，我们还进行了一些关于英语→法语翻译的实验，并且我们发现该表现与英语→德语的表现高度相关，但是在较大的英语→法语数据集上训练模型需要更长的时间。考虑到形态更丰富的德语的翻译也被认为是一项更具挑战性的任务，我们认为使用英语→德语翻译任务来完成超参数扫描是合理的。  

我们使用 Moses2 中的脚本 tokenize 并清理所有数据集，并使用 32,000 次合并操作使用字节对编码（BPE）学习共享子字单位，最终词汇量大约为 37k。我们发现数据预处理会对最终数据产生很大影响，并且由于我们希望实现可重复性，因此我们将数据预处理脚本与 NMT 框架一起发布给公众。有关数据预处理参数的更多详细信息，请参阅读者的代码版本。  

### 3.2 Training Setup and Software  

以下所有实验均使用我们自己的基于 TensorFlow 的软件框架运行（Abadi et al，2016）。我们特意构建了这个框架，以实现神经机器翻译架构的可重现的最先进的实现。作为我们贡献的一部分，我们正在发布重现我们结果所需的框架和所有配置文件。训练在 Nvidia Tesla K40m 和 Tesla K80 GPU 上进行，每个实验分布在 8 个并行 worker 和 6 个参数服务器上。我们使用 128 的批量大小并使用 beam search 进行解码，beam 宽度为 10，并且（Wu等人，2016）中描述了长度归一化惩罚值 0.6。BLEU 分数是使用 Moses3 中的 multi-bleu.perl 脚本对分词数据进行计算的。每个实验最多运行 2.5M steps，并使用不同的初始化复制 4 次。我们每 30 分钟保存一次模型检查点，并根据验证集 BLEU 得分选择最佳检查点。我们报告每个实验的均值和标准差以及最高分数（按照交叉验证）。  

### 3.3 Baseline Model  

根据对以前文献的回顾，我们选择了一个我们知道能够相当好地执行的基线模型。我们的目标是保持基线模型简单和标准，而不是推进start of the art。该模型（2.1中描述）由一个 2 层双向编码器（每个方向 1 层）和一个带乘法（Luong et al。，2015a）注意机制的 2 层解码器组成。我们对编码器和解码器使用 512-unit GRU（Cho等人，2014），并在每个单元的输入处应用 0.2 的 Dropout。我们使用 Adam 优化器进行训练，固定学习率为 0.0001 而不衰减。嵌入维度设置为 512。可以在补充材料中找到所有模型超参数的更详细描述。  

在以下每个实验中，除了正在研究的一个超参数外，基线模型的超参数保持不变。我们希望这可以让我们隔离各种超参数变化的影响。我们认识到此过程不考虑超参数之间的相互作用，并且当我们认为可能发生此类交互时，我们会执行其他实验。  

## 4 Experiments and Discussion  

为了简洁起见，我们只报告平均 BLEU，标准偏差，最佳 BLEU 以及下表中的模型大小。Log 困惑度，tokens/sec 和收敛时间可以在补充材料表中找到。  

### 4.1 Embedding Dimensionality  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/2.jpg?raw=true)

由于词汇量很大，嵌入层可以占模型参数的很大一部分。历史上，研究人员使用 620 维（Bahdanau等，2015）或 1024 维（Luong等，2015a）嵌入。我们预计更大的嵌入会导致更好的 BLEU 分数，或至少更低的困惑，但我们发现并非总是如此。虽然表1 显示 2048 维嵌入产生了总体最好的结果，但他们只改善了一点。即使小 128 维嵌入也表现出色，但收敛速度几乎快两倍。我们发现对小嵌入和大嵌入的梯度更新没有显著差异，并且嵌入矩阵的梯度更新的范数在整个训练中保持大致恒定，而不管大小如何。我们也没有观察到大嵌入的过拟合，并且训练的 log 困惑度在整个实验中大致相同，这表明该模型没有有效地使用额外的参数，并且可能需要更好的优化技术。或者，可能会出现这种情况：具有大嵌入的模型只需要超过250万步就可以收敛到最佳解决方案。  

### 4.2 RNN Cell Variant  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/3.jpg?raw=true)

在 NMT 架构中常常使用 LSTM（Hochreiter和Schmidhuber，1997）和 GRU（Cho等，2014）单元。虽然有一些研究（Greff等，2016）研究了几千例小序列任务中的 cell 变异，但我们并不了解大规模 NMT 环境中的这类研究。  

门控 cells 如 GRU 和 LSTM 的动机是梯度消失问题。使用 vanilla RNN cell，深层网络不能有效地通过多层和时间步骤传播信息和梯度。然而，在基于注意力的模型中，我们认为解码器应该能够几乎完全基于当前输入和注意上下文做出决定，并且我们假设解码器中的 gating 机制不是绝对必要的。这个假设得到以下事实的支持：我们总是将解码器状态初始化为零而不是传递编码器状态，这意味着解码器状态不包含有关编码源的信息。我们通过在解码器中使用 vanilla RNN cell来测试我们的假设。对于 LSTM 和 GRU 变体，我们替换编码器和解码器中的单元。我们使用没有 peephole connections 的 LSTM 细胞，并将 LSTM 和 GRU 细胞的遗忘偏差初始化为1。  

在我们的实验中，LSTM cell 一贯优于 GRU cell。由于我们体系结构中的计算瓶颈是 softmax 操作，因此我们没有观察到 LSTM 和 GRU 单元之间的训练速度差别很大。有些令我们吃惊的是，我们发现 vanilla 解码器无法像门控变体一样学习得很好。这表明解码器确实在多个时间步骤中以其自身状态传递信息，而不是仅仅依赖于注意机制和当前输入（其包括先前的关注上下文）。也可能是门控机制需要掩盖输入中不相关的部分。  

### 4.3 Encoder and Decoder Depth  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/4.jpg?raw=true)

我们通常期望更深层次的网络能够比浅层网络更好地融合（He et al，2016）。尽管一些工作（Luong等，2015b; Zhou等，2016; Luong和Manning，2016; Wu等，2016）已经使用深度网络取得了最先进的成果，但其他人使用浅层网络（Jean等，2015 ; Chung等人，2016; Sennrich等人，2016b）已经取得了类似的结果。因此，不清楚深度的重要性以及浅层网络是否能够产生与深层网络相竞争的结果。在这里，我们探讨了高达 8 层的编码器和解码器深度的影响。对于双向编码器，我们分别在两个方向上堆叠 RNN。例如，Enc-8 模型对应于一个前向和一个后向 4 层 RNN。对于更深层次的网络，我们还试验了两种残差连接方式（He et al，2016），以鼓励梯度流。在标准变体中，如等式（4）所示，我们在连续层之间插入残差连接。如果 $h_t^{(l)}(x_t^{(l)},h_{t-1}^{(l)})$ 是时间步 t 的第 l 层的 RNN 输出，则：  

$x_t^{(l+1)} = h_t^{(l)}(x_t^{(l)},h_{t-1}^{(l)}) + x_t^{(l)}\tag{4}$

其中 $x_t^{(0)}$ 是输入tokens 的词嵌入。 、我们还探索了一种类似于（Huang et al。、，2016a）在图像识别中使用的残差连接的（“ResD”）dense 变体。在这个变体中，我们添加从每个图层到所有其他图层的 skip 连接：  

$x_t^{(l+1)} = h_t^{(l)}(x_t^{(l)},h_{t-1}^{(l)}) + 、sum_{j=0}^l x_t^{(j)}\tag{4}$

我们的实现不同于（Huang et al，2016a），因为我们使用一个加法而不是一个连接操作来保持状态大小不变。  

表3 示出了编码器和解码器不同深度以及具有和不具有残差连接的结果。我们没有找到明确的证据表明编码器深度超过两层是必要的，但发现具有残留连接的深层模型在训练期间更有可能发散。正如大标准偏差所暗示的，最好的深度残差模型取得了良好的结果，但四个运行中只有一个收敛。  

在解码器方面，深层模型的表现略胜一筹，我们发现没有残差连接，我们不可能训练 8 层或更多层的解码器。在深度解码器实验中，密集的残差连接始终优于常规残差连接，并且在步数方面收敛得更快，如图2 所示。我们期望深度模型表现更好（Zhou et al，2016; Szegedy et al， 2015），我们相信我们的实验证明需要更强大的技术来优化深度序列模型。例如，我们可能需要一个更优化的 SGD 优化器或某种形式的批量标准化，以强健地训练具有残差连接的深度网络。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/5.jpg?raw=true)

### 4.4 Unidirectional vs. Bidirectional Encoder  

在文献中，我们使用双向编码器（Bahdanau等，2015），单向编码器（Luong等，2015a）以及两者的混合（Wu等，2016）。双向编码器能够创建考虑过去和未来输入的表示，而单向编码器只能考虑过去的输入。单向编码器的好处是它们的计算可以很容易地在 GPU 上并行化，从而使它们的运行速度比双向同步更快。我们没有意识到任何研究探索双向性的必要性。在这组实验中，我们探索了具有和不具有反向源输入的不同深度的单向编码器，因为这是一种常用的技巧，允许编码器为较早的字创建更丰富的表示。鉴于解码器端的错误可以轻易级联，早期字词的正确性会产生不成比例的影响。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/6.jpg?raw=true)

表4 显示双向编码器通常优于单向编码器，但不会大幅度提高。带有反向信号源的编码器的性能始终优于非反向信号，但不会击败较浅的双向编码器。  

### 4.5 Attention Mechanism  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/7.jpg?raw=true)

两种最常用的注意机制是 additive 变体（Bahdanau等，2015），下面的等式（6），以及计算上较便宜的乘法变体（Luong等，2015a），下面的等式（7）。给定注意 key $h_j$（编码器状态）和注意力查询 $s_i$（解码器状态），每对的注意力分数计算如下：  

$score(h_j,s_i) = \left\langle v,tanh(W_1h_j + W_2s_i) \right\rangle \tag{6}$
$score(h_j,s_i) = \left\langle W_1h_j , W_2s_i \right\rangle \tag{7}$

我们将 W1hj 和 W2si 的维度称为“注意维度”，并通过更改图层大小将其从 128 改为 1024。我们还尝试通过使用最后一个编码器状态（无状态）初始化解码器状态或将最后一个解码器状态连接到每个解码器输入（无输入）来使用无关注机制。结果如表5 所示。  

我们发现参数化加性注意机制略微但一致地优于乘法注意机制，注意维度几乎没有影响。  

尽管我们确实期望基于关注的模型在没有关注机制的情况下显著优于那些模型，但我们对“Non-Input”模型表现差强人意，因为他们在每个时间步可以访问编码器信息。此外，我们发现基于关注的模型在整个训练过程中对解码器状态显示出更大的梯度更新。这表明，注意机制更像是一种“加权跳过连接”，它优化了梯度流程，而不像文献中通常所述的允许编码器访问源状态的“存储器”。我们认为在这个方向上的进一步研究对于阐明关注机制的作用是否有必要，以及它是否可能纯粹是一种简化优化的工具。  

### 4.6 Beam Search Strategies  

Beam Search 是一种常用的技术，通过树搜索来最大化一些评分函数 $s(y,x)$找到目标序列。在最简单的情况下，被最大化的得分是给定源的目标序列的对数概率。最近，诸如覆盖惩罚（Tu等人，2016）和长度归一化（Wu等人，2016）的扩展已经被用来改进解码结果。还观察到（Tu et al，2017），即使长度不足，非常大的 Beam Search 尺寸也比较小的 Beam Search 尺寸差。因此，选择正确的 Beam Search 宽度对获得最佳结果至关重要。  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/8.jpg?raw=true)

表6 显示了改变 Beam Search 宽度和增加长度归一化惩罚的影响。beam width 1 对应于贪婪搜索。我们发现调整好的 Beam Search 对于获得好的结果是至关重要的，并且它会导致不止一个 BLEU 点的一致增益。类似于（Tu et al，2017），我们发现非常大的 Beam 会产生更差的结果，并且存在最佳光束宽度的“最佳点”。我们认为对 Beam Search 中超参数的鲁棒性的进一步研究对 NMT 的进展至关重要。我们还尝试了一个覆盖率惩罚，但没有发现超过足够长的惩罚额外收益。  

### 4.7 Final System Comparison  

![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/9.jpg?raw=true)
![Aaron Swartz](https://github.com/liyibo/cv_notebooks/blob/master/markdown_pics/NLP/6/10.jpg?raw=true)

最后，我们将在 newstest2013 验证集上选择的所有实验（具有512维加性关注的基本模型）的最佳表现模型与表8 中的文献中的历史结果进行比较。虽然不关注此项工作，但通过将我们所有的见解与表7 中描述的单一模型相结合，能够实现进一步的改进。  

尽管我们不提供架构创新，但我们确实通过仔细的超参数调整和良好的初始化来证明，可以在标准 WMT 基准测试中实现最先进的性能。我们的模型仅受到（Wu et al，2016）的表现的影响，该模型更复杂。  

## 5 Open Source Release  

我们凭经验证明了超参数值和不同初始化的微小变化是如何影响结果的，以及如何调整 beam search 等看似微不足道的因素至关重要。为了实现可重复的研究，我们认为研究人员必须开始建立共同的框架和数据处理流程。考虑到这一目标，我们专门构建了一个模块化软件框架，使研究人员能够以最少的代码更改探索新型架构，并以可重现的方式定义实验参数。虽然我们的初始实验是在机器翻译中，但我们的框架可以很容易地适应 Summarization，会话建模或图像到文本中的问题。像 OpenNMT 这样的系统（Klein et al，2017）有相似的目标，但尚未达到最先进的结果（见表8），缺乏我们认为是关键特征的因素，如分布式训练支持。我们希望通过开源我们的实验工具包，使该领域在未来取得更快的进展。  

我们的所有代码均可在 https://github.com/google/seq2seq/ 免费获取。  

## 6 Conclusion  

我们进行了我们认为是神经机器翻译的第一个大规模建筑架构分析，剔除了实现最先进结果的关键因素。我们展示了一些令人惊讶的见解，其中包括 beam search 调整与大多数架构变化同样重要的事实，并且使用当前优化技术，深层模型并不总是胜过浅层。在这里，我们总结了我们的实际发现：  

- 2048尺寸的大型嵌入获得了最佳效果，但只有小幅度的提升。即使 128 维的小嵌入似乎也有足够的能力来捕获大部分必要的语义信息。
- LSTM Cells 始终优于 GRU Cells。
- 具有 2 至 4 层的双向编码器性能最佳。更深层次的编码器对于训练来说显然更加不稳定，但是如果它们能够很好地优化，则显示出潜力。
- 4 层深解码器略胜于更浅的解码器。残差连接对训练 8 层解码器是必要的，密集的残差连接提供了额外的稳健性。
- 参数化的 additive 注意力产生了总体最佳结果。
- 调整良好的附加长度惩罚的波束搜索至关重要。5 到 10 的光束宽度以及 1.0 的长度惩罚似乎工作良好。  

我们强调了几个重要的研究问题，包括有效利用嵌入参数（4.1），注意机制作为加权跳过连接（4.5）而不是内存单元的作用，需要更好的深度循环网络优化方法（4.3），并且需要对超参数变化具有鲁棒性的更好的波束搜索（4.6）。  

此外，我们还向公众发布了一个开放源代码的 NMT 框架，专门用于探索架构创新并生成可重复的实验，以及用于我们所有实验的配置文件。

## 参考  

- 1 [Visualizing and Understanding Neural Models in NLP](https://arxiv.org/abs/1506.01066)  
- 2 [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)  
- 3 [Word2Vec Tutorial](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)  
- 4 [Enriching Word Vectors with Subword Information](https://arxiv.org/abs/1607.04606)  
- 5 [Bag of Tricks for Efficient Text Classification](https://arxiv.org/abs/1607.01759)  
- 6 [Massive Exploration of Neural Machine Translation Architectures](https://arxiv.org/abs/1703.03906)  